In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.2 MB/s eta 0:00:00


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json
import re
import pandas as pd
from datetime import datetime, timedelta
import random
import numpy as np
import gradio as gr

In [5]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [26]:
model_name = "Qwen/Qwen2.5-3B"  # Using a more stable model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [28]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
system_prompt ="""You are a dataset generator assistant. Given a user description, generate dataset specifications in JSON format with Excel-like column types
Supported column types:
- "integer": Whole numbers (with optional min/max range)
- "float": Decimal numbers (with optional min/max range)
- "text": String/text data (with optional categories for categorical data)
- "date": Date values (YYYY-MM-DD format)
- "datetime": Date and time values
- "boolean": True/False values
- "category": Predefined categories/options

Format your response as valid JSON with this structure:
{
  "dataset_name": "descriptive_name",
  "columns": [
    {
      "name": "column_name",
      "type": "column_type",
      "constraints": {
        "min": value (for numeric types),
        "max": value (for numeric types),
        "categories": ["option1", "option2"] (for category type),
        "nullable": true/false
      }
    }
  ]
}
Only respond with valid JSON, no other text."""

In [30]:
def generate_dataset_spec(user_description):
    """Generate dataset specification from user description"""
    prompt = f"{system_prompt}\n\nUser request: {user_description}\n\nJSON:"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Full response: {response}")  # Debug print

    # Find the generated part (after the prompt)
    prompt_end = response.find("JSON:")
    if prompt_end != -1:
        generated_part = response[prompt_end + 5:].strip()  # +5 for "JSON:"
        print(f"Generated part: {generated_part}")  # Debug print
    else:
        generated_part = response

    # Look for all JSON objects in the generated part
    json_objects = []
    i = 0
    while i < len(generated_part):
        if generated_part[i] == '{':
            # Find matching closing brace
            brace_count = 0
            start = i
            for j in range(i, len(generated_part)):
                if generated_part[j] == '{':
                    brace_count += 1
                elif generated_part[j] == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        json_candidate = generated_part[start:j+1]
                        json_objects.append(json_candidate)
                        i = j
                        break
        i += 1

    # Try to parse each JSON object
    for json_str in json_objects:
        try:
            # Clean up the JSON string
            json_str = json_str.strip()
            parsed = json.loads(json_str)
            print(f"Successfully parsed JSON: {parsed}")  # Debug print
            return parsed
        except json.JSONDecodeError as e:
            print(f"Failed to parse JSON candidate: {e}")
            print(f"JSON candidate: {json_str}")
            continue

    print("Failed to parse any JSON. Raw response:", response)
    return None


In [31]:
def generate_sample_data(spec, num_rows=100):
    """Generate sample data based on specification"""
    if not spec:
        print("Error: Dataset specification is empty.")
        return None

    data = {}

    # Check if the specification is in the expected structured format with 'columns' key
    if 'columns' in spec and isinstance(spec['columns'], list):
        columns = spec['columns']
        if not columns:
            print("Warning: 'columns' list is empty in the specification.")
            return None

        for col in columns:
            if not isinstance(col, dict) or 'name' not in col or 'type' not in col:
                print(f"Warning: Skipping invalid column definition: {col}")
                continue

            col_name = col['name']
            col_type = col['type']
            constraints = col.get('constraints', {})
            print(f"Column: {col_name}, Type: {col_type}, Constraints: {constraints}")
            # Pass the constraints directly, no need to re-parse from type string here
            data[col_name] = generate_column_data(col_type, constraints, num_rows)
    else:
        print("Error: Dataset specification is not in the expected structured format with a 'columns' list.")
        print(f"Received spec: {spec}")
        return None # Return None if spec is not in the expected format

    # Check if any data was generated
    if not data:
         print("Error: No data was generated based on the specification.")
         return None

    return pd.DataFrame(data)

In [32]:
def parse_constraints_from_type(col_type, col_name):
    """Parse constraints from column type string"""
    constraints = {}
    col_name_lower = col_name.lower()

    # Smart constraint detection based on column names
    if 'count' in col_name_lower or 'quantity' in col_name_lower:
        constraints = {'min': 1, 'max': 50}
    elif 'id' in col_name_lower:
        constraints = {'min': 1, 'max': 10000}
    elif 'age' in col_name_lower:
        constraints = {'min': 18, 'max': 80}
    elif 'price' in col_name_lower or 'cost' in col_name_lower or 'amount' in col_name_lower:
        constraints = {'min': 10.0, 'max': 1000.0}
    elif 'rating' in col_name_lower or 'score' in col_name_lower:
        constraints = {'min': 1, 'max': 5}
    elif 'name' in col_name_lower and 'product' in col_name_lower:
        constraints = {'categories': ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard', 'Mouse']}
    elif 'name' in col_name_lower and ('pizza' in col_name_lower or 'food' in col_name_lower):
        constraints = {'categories': ['Margherita', 'Pepperoni', 'Hawaiian', 'Veggie Supreme', 'Meat Lovers', 'BBQ Chicken']}
    elif 'department' in col_name_lower:
        constraints = {'categories': ['Sales', 'Engineering', 'Marketing', 'HR', 'Finance', 'Operations']}
    elif 'city' in col_name_lower:
        constraints = {'categories': ['New York', 'London', 'Tokyo', 'Paris', 'Berlin', 'Sydney', 'Mumbai', 'Toronto']}
    elif 'category' in col_name_lower:
        constraints = {'categories': ['Category A', 'Category B', 'Category C', 'Category D']}
    elif 'status' in col_name_lower:
        constraints = {'categories': ['Active', 'Inactive', 'Pending', 'Completed']}

    return constraints


In [36]:
def generate_column_data(col_type, constraints, num_rows):
    """Generate data for a single column"""
    # Ensure col_type is treated as a string for comparison
    col_type = str(col_type).lower()

    if 'integer' in col_type:
        min_val = constraints.get('min', 1)
        max_val = constraints.get('max', 100)
        # Handle None values for min/max gracefully, default to a reasonable range
        if min_val is None:
            min_val = 1
        if max_val is None:
            max_val = 100
        return [random.randint(min_val, max_val) for _ in range(num_rows)]

    elif 'float' in col_type:
        min_val = constraints.get('min', 0.0)
        max_val = constraints.get('max', 100.0)
         # Handle None values for min/max gracefully
        if min_val is None:
            min_val = 0.0
        if max_val is None:
            max_val = 100.0
        return [round(random.uniform(min_val, max_val), 2) for _ in range(num_rows)]

    elif 'text' in col_type or 'category' in col_type: # Treat category and text similarly if categories are provided
        categories = constraints.get('categories')
        if categories and isinstance(categories, list) and categories:
            return [random.choice(categories) for _ in range(num_rows)]
        else:
            # Default text if no categories or invalid categories
            return [f"Value_{i+1}" for i in range(num_rows)]


    elif 'date' in col_type:
        start_date = datetime(2023, 1, 1)
        end_date = datetime(2024, 12, 31)
        date_range = (end_date - start_date).days
        return [(start_date + timedelta(days=random.randint(0, date_range))).strftime('%Y-%m-%d')
                 for _ in range(num_rows)]

    elif 'datetime' in col_type:
        start_date = datetime(2023, 1, 1)
        end_date = datetime(2024, 12, 31)
        date_range = int((end_date - start_date).total_seconds())
        return [(start_date + timedelta(seconds=random.randint(0, date_range))).strftime('%Y-%m-%d %H:%M:%S')
                 for _ in range(num_rows)]

    elif 'boolean' in col_type:
        return [random.choice([True, False]) for _ in range(num_rows)]

    else:
        # Default to text for unknown types
        print(f"Warning: Unknown column type '{col_type}'. Defaulting to text.")
        return [f"Value_{i+1}" for i in range(num_rows)]

In [37]:
def save_dataset(df, filename, format='csv'):
    """Save dataset in specified format"""
    if format.lower() == 'csv':
        df.to_csv(f"{filename}.csv", index=False)
    elif format.lower() == 'excel':
        df.to_excel(f"{filename}.xlsx", index=False)
    elif format.lower() == 'json':
        df.to_json(f"{filename}.json", orient='records', indent=2)
    print(f"Dataset saved as {filename}.{format}")


In [40]:
if __name__ == "__main__":
    # Test with your example
    user_desc = "I want a dataset for pizza sales with ID (integer), Name (text), Date (date), and Count (integer between 1 and 50)."

    print("Generating dataset specification...")
    spec = generate_dataset_spec(user_desc)

    if spec:
        print("Generated specification:")
        print(json.dumps(spec, indent=2))

        # Manual fix for pizza names if needed
        if 'columns' in spec:
            for col in spec['columns']:
                if col['name'].lower() == 'name' and col['type'] == 'text':
                    if 'categories' not in col.get('constraints', {}):
                        col['constraints'] = col.get('constraints', {})
                        col['constraints']['categories'] = ['Margherita', 'Pepperoni', 'Hawaiian', 'Veggie Supreme', 'Meat Lovers', 'BBQ Chicken', 'Four Cheese', 'Mediterranean']
                        print("Added pizza categories to Name column")

        print("\nGenerating sample data...")
        df = generate_sample_data(spec, num_rows=10)  # Generate 10 sample rows

        if df is not None:
            print("\nSample data preview:")
            print(df.head())

            # Save dataset
            save_dataset(df, "pizza_sales_dataset", "csv")
            print(f"\nColumn types in generated DataFrame:")
            print(df.dtypes)
        else:
            print("Failed to generate sample data")
    else:
        print("Failed to generate specification")


Generating dataset specification...
Full response: You are a dataset generator assistant. Given a user description, generate dataset specifications in JSON format with Excel-like column types
Supported column types:
- "integer": Whole numbers (with optional min/max range)
- "float": Decimal numbers (with optional min/max range)
- "text": String/text data (with optional categories for categorical data)
- "date": Date values (YYYY-MM-DD format)
- "datetime": Date and time values
- "boolean": True/False values
- "category": Predefined categories/options

Format your response as valid JSON with this structure:
{
  "dataset_name": "descriptive_name",
  "columns": [
    {
      "name": "column_name",
      "type": "column_type",
      "constraints": {
        "min": value (for numeric types),
        "max": value (for numeric types),
        "categories": ["option1", "option2"] (for category type),
        "nullable": true/false
      }
    }
  ]
}
Only respond with valid JSON, no other text